In [183]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import pprint

f = open('login.txt', 'r')
lines = f.readlines()
#-1 to ignore newline character
CLIENT_ID = lines[0][:-1]
CLIENT_SECRET = lines[1][:-1]
REDIRECT_URI = lines[2][:-1]
username = lines[3]

scope = 'user-top-read user-library-read'

token = util.prompt_for_user_token(
    username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)

sp = spotipy.Spotify(auth=token)

results = sp.current_user_saved_tracks()
tracks = {}
for idx, item in enumerate(results['items']):
    track = item['track']
    print(track['name'])
    tracks[track['id']] = [track['name'], track['artists'][0]['name']]
    # print(idx, track['artists'][0]['name'], "–", track['name'])

# print(tracks)
print(list(tracks.keys())[0])
print()
analysis = sp.audio_analysis(list(tracks.keys())[0])
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(analysis)

Love Love
Test & Recognise - Flume Re-work
Devil Like Me
Gorgeous (& Blanke with Bipolar Sunshine)
Lose My Mind (feat. Bipolar Sunshine)
Father And Son
As the World Caves In
A First Date
Stuck In The Middle With You
In Bloom
Good Grief
Life Goes On
Sweater Weather
Glue
Two Moons
FEEL LIKE
fue mejor (feat. SZA)
Don't Let Me Down (feat. Daya) - Illenium Remix
San Frandisco
Sonnentanz - Sun Don't Shine
6bCRq6dymqY47l4uEjS0Pc

{ 'bars': [ {'confidence': 0.35, 'duration': 1.96075, 'start': 2.24549},
            {'confidence': 0.413, 'duration': 1.96172, 'start': 4.20624},
            {'confidence': 0.134, 'duration': 1.97207, 'start': 6.16797},
            {'confidence': 0.388, 'duration': 1.9853, 'start': 8.14004},
            {'confidence': 0.132, 'duration': 1.96222, 'start': 10.12534},
            {'confidence': 0.311, 'duration': 1.96496, 'start': 12.08756},
            {'confidence': 0.161, 'duration': 1.96031, 'start': 14.05252},
            {'confidence': 0.618, 'duration': 1.97354,

                              -39.364,
                              61.009,
                              29.838,
                              -15.772,
                              -15.435,
                              -4.121,
                              7.665,
                              9.845,
                              -6.715]},
                { 'confidence': 0.919,
                  'duration': 0.46399,
                  'loudness_end': 0.0,
                  'loudness_max': -4.799,
                  'loudness_max_time': 0.06444,
                  'loudness_start': -16.694,
                  'pitches': [ 0.179,
                               0.639,
                               0.079,
                               0.2,
                               0.162,
                               0.307,
                               0.235,
                               0.281,
                               1.0,
                               0.442,
                           

                  'timbre': [ 55.438,
                              71.052,
                              7.727,
                              -29.552,
                              29.319,
                              -48.257,
                              48.255,
                              19.248,
                              2.974,
                              -9.794,
                              -13.456,
                              13.495]},
                { 'confidence': 0.125,
                  'duration': 0.32549,
                  'loudness_end': 0.0,
                  'loudness_max': -2.351,
                  'loudness_max_time': 0.20301,
                  'loudness_start': -5.412,
                  'pitches': [ 0.778,
                               1.0,
                               0.536,
                               0.547,
                               0.366,
                               0.266,
                               0.133,
                          

                  'timbre': [ 48.319,
                              92.283,
                              -97.998,
                              -38.206,
                              70.769,
                              27.775,
                              12.119,
                              -32.779,
                              -13.175,
                              37.907,
                              -21.596,
                              11.982]},
                { 'confidence': 0.691,
                  'duration': 0.24145,
                  'loudness_end': 0.0,
                  'loudness_max': -7.535,
                  'loudness_max_time': 0.03766,
                  'loudness_start': -16.205,
                  'pitches': [ 0.377,
                               0.284,
                               0.1,
                               0.104,
                               0.34,
                               1.0,
                               0.287,
                        

                               0.116],
                  'start': 54.71206,
                  'timbre': [ 52.666,
                              89.768,
                              86.145,
                              -3.389,
                              67.136,
                              -35.501,
                              -33.458,
                              -6.623,
                              28.597,
                              6.593,
                              -2.822,
                              -4.794]},
                { 'confidence': 0.772,
                  'duration': 0.36549,
                  'loudness_end': 0.0,
                  'loudness_max': -2.894,
                  'loudness_max_time': 0.01778,
                  'loudness_start': -16.061,
                  'pitches': [ 0.129,
                               1.0,
                               0.306,
                               0.044,
                               0.041,
                         

...

                               0.04,
                               0.109,
                               0.045,
                               0.092,
                               0.094,
                               0.345,
                               1.0,
                               0.224,
                               0.063,
                               0.056],
                  'start': 96.88395,
                  'timbre': [ 55.183,
                              28.835,
                              6.714,
                              -14.368,
                              39.443,
                              -42.236,
                              -10.135,
                              8.941,
                              2.915,
                              22.864,
                              -1.841,
                              -10.179]},
                { 'confidence': 0.718,
                  'duration': 0.31524,
                  'loudness_end': 0.0,
         

                  'pitches': [ 0.53,
                               0.44,
                               0.413,
                               0.97,
                               0.536,
                               0.95,
                               0.506,
                               0.391,
                               1.0,
                               0.328,
                               0.377,
                               0.487],
                  'start': 115.4322,
                  'timbre': [ 54.821,
                              75.668,
                              9.967,
                              -22.758,
                              50.221,
                              -22.31,
                              -33.127,
                              -2.594,
                              2.386,
                              3.402,
                              -16.569,
                              -17.267]},
                { 'confidence': 0.207,
              

                               0.164,
                               0.248,
                               0.188,
                               0.305,
                               0.209],
                  'start': 127.50621,
                  'timbre': [ 53.506,
                              -18.051,
                              -19.064,
                              12.766,
                              1.86,
                              -72.73,
                              -1.457,
                              -8.331,
                              -13.199,
                              -3.876,
                              -10.899,
                              14.348]},
                { 'confidence': 0.253,
                  'duration': 0.21374,
                  'loudness_end': 0.0,
                  'loudness_max': -4.174,
                  'loudness_max_time': 0.00861,
                  'loudness_start': -10.174,
                  'pitches': [ 0.974,
                     

                  'loudness_max': -1.414,
                  'loudness_max_time': 0.04827,
                  'loudness_start': -6.561,
                  'pitches': [ 0.232,
                               0.197,
                               0.099,
                               0.462,
                               0.063,
                               0.045,
                               0.068,
                               0.198,
                               1.0,
                               0.149,
                               0.186,
                               0.111],
                  'start': 140.13223,
                  'timbre': [ 56.114,
                              69.384,
                              9.253,
                              -25.228,
                              64.809,
                              -25.197,
                              18.712,
                              6.291,
                              -3.131,
                              7

                              -16.473,
                              24.581,
                              5.797,
                              0.165,
                              -5.3,
                              11.787]},
                { 'confidence': 0.723,
                  'duration': 0.1346,
                  'loudness_end': 0.0,
                  'loudness_max': -5.547,
                  'loudness_max_time': 0.08481,
                  'loudness_start': -12.943,
                  'pitches': [ 0.453,
                               1.0,
                               0.366,
                               0.254,
                               0.576,
                               0.578,
                               0.601,
                               0.618,
                               0.411,
                               0.309,
                               0.245,
                               0.029],
                  'start': 145.93076,
                  'timbre': [

In [96]:
import sqlite3
# con = sqlite3.connect('songs.db')

# cur = con.cursor()
# cur.execute('''drop table if exists tracks''')
# cur.execute(
#     '''create table tracks 
#         (acousticness real not null,
#         artists text not null,
#         danceability real not null,
#         duration_ms real not null,
#         energy real not null,
#         id text not null,
#         instrumentalness real not null,
#         key int not null,
#         liveness real not null,
#         loudness real not null,
#         mode int not null,
#         name text not null,
#         speechiness real not null,
#         tempo real not null,
#         valence real not null,
#         primary key(id))''')
# con.commit()
# con.close()

In [151]:
import pandas as pd
import os
import json
import time

path = '/Users/nickparis/spotify-rec/spotify_million_playlist_dataset/data/'
files = os.listdir(path)

con = sqlite3.connect('songs.db')
cur = con.cursor()

f = files.index('mpd.slice.946000-946999.json')
print(f)
print(len(files))
# already got first few
files = files[f:]
for file_idx in range(len(files)-f):
    file = files[file_idx]
    print('reading file ' + file)
    data = json.load(open(path+file))
    data = data['playlists']
#     pp.pprint(data)

    if type(data) is not list:
        data = list(data)

    # build object to insert into database
    errs = 0
    for playlist in data:
        # get all track ids from playlist
        #ignore spotify:track:
        ids = [track['track_uri'][14:] for track in playlist['tracks']]
        names = [track['track_name'] for track in playlist['tracks']]
        names = [name.replace("'", "") for name in names]
        artists = [track['artist_name'] for track in playlist['tracks']]
        artists = [artist.replace("'", "") for artist in artists]
        
        try:
            # spotify API supports only 100IDs at once
            while True:
                ids_shortened = ids[0:100]
                names_shortened = names[0:100]
                artists_shortened = artists[0:100]

                features = sp.audio_features(tracks=ids_shortened)
    #             pp = pprint.PrettyPrinter(indent=1)
    #             pp.pprint(features)
                while None in features:
                    idx = features.index(None)
                    features.pop(idx)
                    names_shortened.pop(idx)
                    artists_shortened.pop(idx)


                # populate dataframe for easier insert into db
                df = pd.json_normalize(features)
                df['name'] = names_shortened
                df['artists'] = artists_shortened
                for _, row in df.iterrows():
                    query = \
                    '''INSERT INTO tracks ''' +\
                    '''(acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,name ,speechiness,tempo,valence)''' +\
                    ''' VALUES ''' +\
                    '''({},{},{},{},{},{},{},{},{},{},{},{},{},{},{});'''\
                    .format(row['acousticness'],'\''+row['artists']+'\'',row['danceability'],row['duration_ms'],row['energy'],'\''+row['id']+'\'',row['instrumentalness'],row['key'],row['liveness'],row['loudness'],row['mode'],'\''+row['name']+'\'' ,row['speechiness'],row['tempo'],row['valence'])
                    try:
                        cur.execute(query)
                    except sqlite3.Error as err:
                        errs += 1
                        if errs == 50000:
                            print('50000 insert errors')
                            print(str(err) + ' ' + row['name'])
                            errs = 0

                con.commit()

                ids = ids[100:]
                names = names[100:]
                artists = artists[100:]

                if len(ids) == 0:
                    break
        except spotipy.client.SpotifyException:
            #refresh token
            token = util.prompt_for_user_token(
    username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)

            sp = spotipy.Spotify(auth=token)
            file_idx -= 1
        except Exception as e:
            print('error')
            print(e)
            print('playlist ' + playlist['name'])
            print('file ' + file)
con.close()
    
    


68
1001
reading file mpd.slice.946000-946999.json


KeyboardInterrupt: 

In [169]:
#identify duplicates

con = sqlite3.connect('songs.db')
query = '''select t1.artists, t1.name, t1.id from tracks t1 inner join ''' + \
    '''tracks t2 on t1.artists=t2.artists and t1.name=t2.name and t1.id<>t2.id order by t1.name'''

dupes = pd.read_sql_query(query, con)
dupes

,artists,name,id
0,Nick Drake,Cello Song,6p6ocbEBSBOswTLG9xSToK
1,Nick Drake,Cello Song,6ab6cQ8CtAoM9nafkEJEZs
2,Eddie Murphy,"""Buckwheat""",4Li1CTGmeRr0utBexNj7nY
3,Eddie Murphy,"""Buckwheat""",00HoZVs6yTHp6hpt0gLc5m
4,Tylman Susato,"""Danserye"" From Het Derde Musijck Boexcken",6551rVcUfxtyIblfeVaWJu
...,...,...,...
81955,EXO,늑대와 미녀 Wolf - Live,4xJx8JihEl6AEeuKc4vfrX
81956,JJ Lin,사랑해요只對你說,3Sv4RD58BLEd0OvPVBp1vg
81957,JJ Lin,사랑해요只對你說,2rvNd2qPkq056yZBbAAfaS
81958,Standing Egg,오래된 노래,5gllIJSLyRouz1bGO5ues4


now we will remove the duplicates

In [177]:
remove = dupes.iloc[::2,:]
cur = con.cursor()
for i in range(remove.shape[0]):
    query = '''delete from tracks where id=\'{}\''''.format(remove.iloc[i]['id'])
    cur.execute(query)
con.commit()
cur.close()

In [178]:
con = sqlite3.connect('songs.db')
cur = con.cursor()
query = '''select * from tracks'''
data = pd.read_sql_query(query, con)
data.memory_usage(deep=True).sum()

385781174

we have ~386MB of song data. Let's save the dataframe in a csv

In [179]:
data.to_csv('data.csv', index=False)

In this file, we make use of the json formatted million playlist song data. For each song, we hit the spotipy api to get audio_features, and throw that information into a sqlite database in case my computer can not handle the memory restrictions. After running about 10 hours, we have 1.1 million songs. This is about half of the 2 million + songs that are listed on the website.

We then dump the song data into a csv which is 154 MB according to what my mac finder says. In the next file we will begin data exploration and analysis.

references
https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge

https://spotipy.readthedocs.io/en/2.19.0/